In [49]:
# modelling
import pandas as pd
import numpy as np
import matplotlib.cm as cm

racing = pd.read_csv('data.csv')

In [31]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from dataset import Dataset 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold 

validRacingLaps = racing[racing['INVALID_LAP'] == 0]
'''
X=validRacingLaps.head(2)[['FB_BRAKE', 'MB_BRAKE', 'FT_THROTTLE', 'MT_THROTTLE', 'FS_STEERING', 'FS_BRAKE', 'MS_STEERING', 'MS_BRAKE',
                   'FB_SPEED', 'FT_SPEED', 'MT_SPEED', 'FS_SPEED', 'MS_SPEED',
                  'FB_DIST_APEX_1', 'FS_DIST_APEX_1', 'FT_DIST_APEX_1', 'MS_DIST_APEX_1', 'MS_DIST_APEX_2', 'MB_DIST_APEX_1', 'MT_DIST_APEX_1',
                  'MT_DIST_APEX_2',
                   'FS_DIST_FROM_LEFT', 'FT_DIST_FROM_LEFT', 'MS_DIST_FROM_LEFT', 'MT_DIST_FROM_LEFT',
                  'FB_ANGLE_APEX_1', 'MB_ANGLE_APEX_1', 'FS_ANGLE_APEX_1', 'MS_ANGLE_APEX_1', 'MS_ANGLE_APEX_2', 'FT_ANGLE_APEX_1', 'FT_ANGLE_APEX_2',
                  'MT_ANGLE_APEX_1', 'MT_ANGLE_APEX_2',
                  'FB_LAP_DIST', 'MB_LAP_DIST', 'FS_LAP_DIST', 'MS_LAP_DIST', 'FT_LAP_DIST', 'MT_LAP_DIST']]
'''
X=validRacingLaps.head(2)[['FB_BRAKE', 'MB_BRAKE', 'FT_THROTTLE', 'MT_THROTTLE', 'FS_STEERING', 'FS_BRAKE', 'MS_STEERING', 'MS_BRAKE', 'MS_THROTTLE',
                   'FB_SPEED', 'FT_SPEED', 'MT_SPEED', 'FS_SPEED', 'MS_SPEED',
                  'FB_LAP_DIST', 'FS_LAP_DIST', 'FT_LAP_DIST', 'MS_LAP_DIST', 'MS_LAP_DIST', 'MB_LAP_DIST', 'MT_LAP_DIST',
                   'FB_DIST_FROM_LEFT', 'FS_DIST_FROM_LEFT', 'FT_DIST_FROM_LEFT', 'MS_DIST_FROM_LEFT', 'MT_DIST_FROM_LEFT',
                   '360_DIST_FROM_LEFT', '360_ANGLE_APEX_1', '360_SPEED', '468_DIST_FROM_LEFT', '468_ANGLE_APEX_2', '468_SPEED']]

y=validRacingLaps.head(2)['SECTION_TIME_MS']

#validRacingLaps2Rows = X.head(2) 
#validRacingLapsY = X.head(2).assign(SECTION_TIME_MS=y.head(2).values) 

In [32]:
#baseline model
crossValidation = KFold(n_splits=2, shuffle=True, random_state=1)

for depth in range(1, 10): 
    tree_regressor=tree.DecisionTreeRegressor(max_depth=depth, random_state=1) 
    if tree_regressor.fit(X,y).tree_.max_depth<depth:
        break
    score=np.mean(cross_val_score(tree_regressor, X, y,
                                 scoring='neg_mean_squared_error',
                                 cv=crossValidation, n_jobs=1))
    print(depth, score) 
# lowest = max depth of 3 for score of -1310778.610073403

1 -36481.0


In [33]:
search.fit(X,y)
print(search.best_params_)
print(search.best_score_)

{'learning_rate': 0.001, 'max_depth': 1, 'n_estimators': 500, 'random_state': 1, 'subsample': 1}
-36481.0


In [34]:
# best params
best_params = search.best_params_
model = GradientBoostingRegressor(
    n_estimators=best_params['n_estimators'], 
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    random_state=best_params['random_state'],
    subsample=best_params['subsample']
)

X=validRacingLaps.head(2)[['FB_BRAKE', 'MB_BRAKE', 'FT_THROTTLE', 'MT_THROTTLE', 'FS_STEERING', 'FS_BRAKE', 'MS_STEERING', 'MS_BRAKE', 'MS_THROTTLE',
                   'FB_SPEED', 'FT_SPEED', 'MT_SPEED', 'FS_SPEED', 'MS_SPEED',
                  'FB_LAP_DIST', 'FS_LAP_DIST', 'FT_LAP_DIST', 'MS_LAP_DIST', 'MS_LAP_DIST', 'MB_LAP_DIST', 'MT_LAP_DIST',
                   'FB_DIST_FROM_LEFT', 'FS_DIST_FROM_LEFT', 'FT_DIST_FROM_LEFT', 'MS_DIST_FROM_LEFT', 'MT_DIST_FROM_LEFT',
                   '360_DIST_FROM_LEFT', '360_ANGLE_APEX_1', '360_SPEED', '468_DIST_FROM_LEFT', '468_ANGLE_APEX_2', '468_SPEED']]

y=validRacingLaps.head(2)['SECTION_TIME_MS']

model.fit(X,y)
predictions=model.predict(X)
results=pd.DataFrame({'Actual': y, 'Predicted': predictions})
print(results.head())

from sklearn.metrics import mean_squared_error 
mse = mean_squared_error(y, predictions)
print(f"Mean Squared Error: {mse}")

     Actual     Predicted
10  10976.0  11033.909189
14  11167.0  11109.090811
Mean Squared Error: 3353.474197766424


In [36]:
# Lap distance regression 
#find fastest laps - top 20%
fastest_laps = validRacingLaps.sort_values(by='SECTION_TIME_MS').head(int(len(validRacingLaps) * 0.2))

In [113]:
#train regression on it
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error 

X = fastest_laps[['FB_LAP_DIST', 'MB_LAP_DIST', 'FS_LAP_DIST', 'MS_LAP_DIST', 'FT_LAP_DIST', 'MT_LAP_DIST']]
y = fastest_laps['SECTION_TIME_MS']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)

y_pred = ridge_model.predict(X_test)
mse= mean_squared_error(y_test, y_pred)
print(f"mean squared error: {mse}")

mean squared error: 6297.904671749915


In [202]:
from scipy.optimize import minimize 
import numpy as np 

# using optimiser to optimise LAP_DIST for key moments such that SECTION_TIME_MS is minimised 

X = validRacingLaps[['FB_LAP_DIST', 'MB_LAP_DIST', 'FS_LAP_DIST', 'MS_LAP_DIST', 'FT_LAP_DIST', 'MT_LAP_DIST']]
y = validRacingLaps['SECTION_TIME_MS']
model = GradientBoostingRegressor(random_state=1)
model.fit(X, y)

def objective(lap_dists):
    lap_dist_features = np.array(lap_dists).reshape(1, -1)
    predicted_time = model.predict(lap_dist_features)
    return predicted_time[0]

# different ways of initialising 
# mean
init_mean = X.mean().values 
method = 'Nelder-Mead'
result_init_mean = minimize(objective, init_mean, method=method)
optimal_lap_dists_init_mean = result_init_mean.x
predicted_min_time_init_mean = result_init_mean.fun 

print(f"Optimal LAP_DIST configuration for minimum SECTION_TIME_MS for mean initial values, method = {method}:")
for moment, opt_dist in zip(['FB_LAP_DIST', 'MB_LAP_DIST', 'FS_LAP_DIST', 'MS_LAP_DIST', 'FT_LAP_DIST', 'MT_LAP_DIST'], optimal_lap_dists_init_mean):
    print(f"{moment}: {opt_dist:.2f}")
print(f"Predicted minimum SECTION_TIME_MS: {predicted_min_time_init_mean}")
print("\n")

# median
init_median = X.median().values
result_init_median = minimize(objective, init_median, method=method)
optimal_lap_dists_init_median = result_init_median.x
predicted_min_time_init_median = result_init_median.fun

print(f"Optimal LAP_DIST configuration for minimum SECTION_TIME_MS for median initial values, method = {method}:")
for moment, opt_dist in zip(['FB_LAP_DIST', 'MB_LAP_DIST', 'FS_LAP_DIST', 'MS_LAP_DIST', 'FT_LAP_DIST', 'MT_LAP_DIST'], optimal_lap_dists_init_median):
    print(f"{moment}: {opt_dist:.2f}")
print(f"Predicted minimum SECTION_TIME_MS: {predicted_min_time_init_median}")
print("\n")

# random sampling 
# est ranges
lap_dist_ranges = {
    'FB_LAP_DIST': (8.075573921203613, 351.6825561523437),
    'FS_LAP_DIST': (11.171208381652832, 370.0),
    'MB_LAP_DIST': (27.187135696411133, 440.8147888183594),
    'FT_LAP_DIST': (12.92138671875, 449.99908447265625),
    'MS_LAP_DIST': (33.651493072509766, 414.9357299804687),
    'MT_LAP_DIST': (31.827800750732425, 765.2254028320312)
}

init_random = np.array([
    np.random.uniform(low, high) for low, high in lap_dist_ranges.values()
])
#print("Randomly sampled initial values for optimization:", init_random)

result_init_random = minimize(
    objective, init_random, method='Nelder-Mead',
    bounds=[lap_dist_ranges[key] for key in lap_dist_ranges.keys()] 
)
optimal_lap_dists_init_random = result_init_random.x
predicted_min_time_init_random = result_init_random.fun 

print(f"Optimal LAP_DIST configuration for minimum SECTION_TIME_MS for random initial values, method = {method}:")
for moment, opt_dist in zip(['FB_LAP_DIST', 'MB_LAP_DIST', 'FS_LAP_DIST', 'MS_LAP_DIST', 'FT_LAP_DIST', 'MT_LAP_DIST'], optimal_lap_dists_init_random):
    print(f"{moment}: {opt_dist:.2f}")
print(f"Predicted minimum SECTION_TIME_MS: {predicted_min_time_init_random}")
print("\n")


Optimal LAP_DIST configuration for minimum SECTION_TIME_MS for mean initial values, method = Nelder-Mead:
FB_LAP_DIST: 279.91
MB_LAP_DIST: 316.73
FS_LAP_DIST: 325.76
MS_LAP_DIST: 386.06
FT_LAP_DIST: 335.68
MT_LAP_DIST: 434.48
Predicted minimum SECTION_TIME_MS: 10747.328627827206


Optimal LAP_DIST configuration for minimum SECTION_TIME_MS for median initial values, method = Nelder-Mead:
FB_LAP_DIST: 268.71
MB_LAP_DIST: 317.21
FS_LAP_DIST: 320.83
MS_LAP_DIST: 388.18
FT_LAP_DIST: 340.45
MT_LAP_DIST: 421.89
Predicted minimum SECTION_TIME_MS: 10736.76292572451


Optimal LAP_DIST configuration for minimum SECTION_TIME_MS for random initial values, method = Nelder-Mead:
FB_LAP_DIST: 259.66
MB_LAP_DIST: 75.72
FS_LAP_DIST: 203.08
MS_LAP_DIST: 256.95
FT_LAP_DIST: 60.97
MT_LAP_DIST: 41.53
Predicted minimum SECTION_TIME_MS: 25782.75603716743




In [273]:
from scipy.optimize import minimize 
import numpy as np 

# using fixed lap dist, optimise driver inputs at each key moment .
driver_input_cols = [col for col in X_model2.columns if any(x in col for x in ['_THROTTLE', '_BRAKE', '_STEERING'])]

fixed_lap_dist = {
    'FB_LAP_DIST': 268.71,
    'MB_LAP_DIST': 317.21,
    'FS_LAP_DIST': 320.83,
    'MS_LAP_DIST': 256.95,
    'FT_LAP_DIST': 340.45,
    'MT_LAP_DIST': 421.89    
} 

steering_bounds = (-1, 1)
throttle_bounds = (0, 1)
brake_bounds = (0, 1)
bounds = []
for col in driver_input_cols:
    if 'STEERING' in col:
        bounds.append(steering_bounds)
    elif 'THROTTLE' in col:
        bounds.append(throttle_bounds)
    elif 'BRAKE' in col:
        bounds.append(brake_bounds)

X_model2 = validRacingLaps[[
     'FB_THROTTLE', 'FB_BRAKE',
    'MB_STEERING', 'MB_BRAKE', 
    'FS_STEERING', 'FS_BRAKE', 
    'MS_STEERING', 'MS_THROTTLE', 'MS_BRAKE',
    'FT_STEERING', 'FT_THROTTLE', 'FT_BRAKE', 
    'MT_STEERING', 'MT_THROTTLE'
]]

y_model2 = validRacingLaps['SECTION_TIME_MS']
model = GradientBoostingRegressor(random_state=1)
model.fit(X_model2, y_model2)

def objective(lap_dists):
    lap_dist_features = np.array(lap_dists).reshape(1, -1)
    predicted_time = model.predict(lap_dist_features)
    return predicted_time[0]

# initialise driver inputs as those of a random driver in the top5 fastest laps 
init_driver_inputs_mean = X_model2.mean().values

method = 'COBYQA'
result_model2 = minimize(objective, init_driver_inputs_mean, method=method, bounds=bounds)
optimal_features_model2 = result_model2.x
predicted_time_model2 = result_model2.fun

print(f"Optimal driver inputs for minimum SECTION_TIME_MS with fixed LAP_DIST values, init = driver input means, method = {method}:")
for moment, opt_value in zip(driver_input_cols, optimal_features_model2):
    print(f"{moment}: {opt_value:.2f}")
print(f"Predicted minimum SECTION_TIME_MS: {predicted_time_model2}")

Optimal driver inputs for minimum SECTION_TIME_MS with fixed LAP_DIST values, init = driver input means, method = COBYQA:
FB_THROTTLE: 0.70
FB_BRAKE: 0.75
MB_STEERING: -0.43
MB_BRAKE: 0.88
FS_STEERING: 0.16
FS_BRAKE: 1.00
MS_STEERING: 0.37
MS_THROTTLE: 0.71
MS_BRAKE: 0.00
FT_STEERING: 0.32
FT_THROTTLE: 0.00
FT_BRAKE: 0.04
MT_STEERING: 0.67
MT_THROTTLE: 1.00
Predicted minimum SECTION_TIME_MS: 10099.386089674184


In [281]:
from scipy.optimize import minimize 
import numpy as np 

# initialise using top 5 fastest drivers
driver_input_cols = [
    'FB_THROTTLE', 'FB_BRAKE',
    'MB_STEERING', 'MB_BRAKE', 
    'FS_STEERING', 'FS_BRAKE', 
    'MS_STEERING', 'MS_THROTTLE', 'MS_BRAKE',
    'FT_STEERING', 'FT_THROTTLE', 'FT_BRAKE', 
    'MT_STEERING', 'MT_THROTTLE'
]

fixed_lap_dist = {
    'FB_LAP_DIST': 268.71,
    'MB_LAP_DIST': 317.21,
    'FS_LAP_DIST': 320.83,
    'MS_LAP_DIST': 256.95,
    'FT_LAP_DIST': 340.45,
    'MT_LAP_DIST': 421.89    
} 

steering_bounds = (-1, 1)
throttle_bounds = (0, 1)
brake_bounds = (0, 1)
bounds = []
for col in driver_input_cols:
    if 'STEERING' in col:
        bounds.append(steering_bounds)
    elif 'THROTTLE' in col:
        bounds.append(throttle_bounds)
    elif 'BRAKE' in col:
        bounds.append(brake_bounds)

top_5_fastest_laps = validRacingLaps.nsmallest(5, 'SECTION_TIME_MS')
random_lap = top_5_fastest_laps.sample(n=1).iloc[0]
initial_driver_inputs_random = random_lap[driver_input_cols].values

X_model2 = validRacingLaps[driver_input_cols]
y_model2 = validRacingLaps['SECTION_TIME_MS']
model = GradientBoostingRegressor(random_state=1)
model.fit(X_model2, y_model2)

def objective(driver_inputs):
    lap_dist_features = np.array(driver_inputs).reshape(1, -1)
    predicted_time = model.predict(lap_dist_features)
    return predicted_time[0]

method = 'COBYLA'
result_model3 = minimize(objective, initial_driver_inputs_random, method=method, bounds=bounds)
optimal_features_model3 = result_model3.x
predicted_time_model3 = result_model3.fun

print(f"Optimal driver inputs for minimum SECTION_TIME_MS with fixed LAP_DIST values, init = randomly selected top 5 driver, method = {method}:")
for moment, opt_value in zip(driver_input_cols, optimal_features_model3):
    print(f"{moment}: {opt_value:.2f}")
print(f"Predicted minimum SECTION_TIME_MS: {predicted_time_model3}")

Optimal driver inputs for minimum SECTION_TIME_MS with fixed LAP_DIST values, init = randomly selected top 5 driver, method = COBYLA:
FB_THROTTLE: 1.00
FB_BRAKE: 1.00
MB_STEERING: -0.52
MB_BRAKE: 0.96
FS_STEERING: 1.00
FS_BRAKE: 1.00
MS_STEERING: 0.40
MS_THROTTLE: 0.58
MS_BRAKE: 0.01
FT_STEERING: 0.34
FT_THROTTLE: 0.00
FT_BRAKE: 0.05
MT_STEERING: 1.00
MT_THROTTLE: 1.00
Predicted minimum SECTION_TIME_MS: 10038.798716685122


In [114]:
for feature, coef in zip(X.columns, ridge_model.coef_):
    print(f"{feature}:{coef}")

FB_LAP_DIST:-1.0386460851370698
MB_LAP_DIST:0.5443339749082587
FS_LAP_DIST:6.981561535887458
MS_LAP_DIST:-0.27086670837279275
FT_LAP_DIST:2.5936340476155313
MT_LAP_DIST:0.027545096178468124


In [115]:
# approximate optimal values - adjust values to predict lap time. 
sample_lap = X.median().to_frame().T 
sample_lap['FB_LAP_DIST'] = 150 
predicted_time = ridge_model.predict(sample_lap)
print(f"predicted SECTION_TIME_MS with adjusted LAP_DIST: {predicted_time[0]}")

predicted SECTION_TIME_MS with adjusted LAP_DIST: 10799.99284699909


In [57]:
# optimising key moments to reduce SECTION_TIME_MS 
# ranges taken from the top 20% of laps - imputed custom range for MT 
# due to excessive wide range + outliers - used IQR with threshold 3.5 in conjunction with qualitative observations about the concentration 
# where MT occurs the most on the track. 

X = validRacingLaps[['FB_LAP_DIST', 'FS_LAP_DIST', 'MB_LAP_DIST', 'FT_LAP_DIST', 'MS_LAP_DIST', 'MT_LAP_DIST']]
y = validRacingLaps['SECTION_TIME_MS']

ranges = {
    'FB_LAP_DIST': (245.46, 292.98),
    'FS_LAP_DIST': (313.00, 336.72),
    'MB_LAP_DIST': (256.51, 352.60),
    'FT_LAP_DIST': (325.00, 384.31),
    'MS_LAP_DIST': (346.57, 414.01),
    'MT_LAP_DIST': (360, 700)
}

from sklearn.model_selection import ParameterGrid
param_grid = {
    'FB_LAP_DIST': np.linspace(*ranges['FB_LAP_DIST'], num=10),
    'FS_LAP_DIST': np.linspace(*ranges['FS_LAP_DIST'], num=10),
    'MB_LAP_DIST': np.linspace(*ranges['MB_LAP_DIST'], num=10),
    'FT_LAP_DIST': np.linspace(*ranges['FT_LAP_DIST'], num=10),
    'MS_LAP_DIST': np.linspace(*ranges['MS_LAP_DIST'], num=10),
    'MT_LAP_DIST': np.linspace(*ranges['MT_LAP_DIST'], num=10),
}

best_time = float('inf')
best_params = None
grid = list(ParameterGrid(param_grid))
model = GradientBoostingRegressor(n_estimators=500, learning_rate=0.01, max_depth=4, random_state=1)
model.fit(X, y)

GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=500,
                          random_state=1)

In [58]:
for params in grid:
    test_X = np.array([params['FB_LAP_DIST'], params['FS_LAP_DIST'], params['MB_LAP_DIST'],
                       params['FT_LAP_DIST'], params['MS_LAP_DIST'], params['MT_LAP_DIST']]).reshape(1, -1)
    
    predicted_time = model.predict(test_X)[0]
    if predicted_time < best_time:
        best_time = predicted_time
        best_params = params

print("Best LAP_DIST configuration for minimum SECTION_TIME_MS:")
print(best_params)
print("Predicted SECTION_TIME_MS:", best_time)

Best LAP_DIST configuration for minimum SECTION_TIME_MS:
{'FB_LAP_DIST': np.float64(261.3), 'FS_LAP_DIST': np.float64(318.2711111111111), 'FT_LAP_DIST': np.float64(331.59), 'MB_LAP_DIST': np.float64(288.54), 'MS_LAP_DIST': np.float64(376.54333333333335), 'MT_LAP_DIST': np.float64(360.0)}
Predicted SECTION_TIME_MS: 10815.807959055604
